# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

# Good Luck!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('tree-exercise').getOrCreate()

In [3]:
data = spark.read.csv('../data/dog_food.csv', inferSchema=True, header=True)

In [4]:
data.printSchema()
data.describe().show()
data.show(5)

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: integer (nullable = true)

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|                 A|                 B|                 C|                 D|            Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               490|               490|               490|               490|                490|
|   mean|  5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
| stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
|    min|                 1|                 1|               5.0|                 1|                  0|
|    max|                10|                10|              14.0|           

In [5]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [6]:
train_data, test_data = data.randomSplit([.8, .2], seed=42)

In [7]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')

train_data = assembler.transform(train_data)
test_data = assembler.transform(test_data)

train_data.show(5)

+---+---+----+---+-------+------------------+
|  A|  B|   C|  D|Spoiled|          features|
+---+---+----+---+-------+------------------+
|  1|  1|10.0|  8|      1|[1.0,1.0,10.0,8.0]|
|  1|  1|12.0|  2|      1|[1.0,1.0,12.0,2.0]|
|  1|  1|13.0|  3|      1|[1.0,1.0,13.0,3.0]|
|  1|  2| 9.0|  1|      0| [1.0,2.0,9.0,1.0]|
|  1|  2| 9.0|  4|      0| [1.0,2.0,9.0,4.0]|
+---+---+----+---+-------+------------------+
only showing top 5 rows



In [8]:
rfc = RandomForestClassifier(featuresCol='features', labelCol='Spoiled', seed=42)
rfc_model = rfc.fit(train_data)
rfc_result = rfc_model.evaluate(test_data)


In [9]:
print(f'Accuracy: {rfc_result.accuracy}')
print(f'AUC: {rfc_result.areaUnderROC}')
print(f'F1: {rfc_result.fMeasureByLabel()}')
print(f'Precision: {rfc_result.precisionByLabel}')
print(f'Recall: {rfc_result.recallByLabel}')

Accuracy: 0.9863013698630136
AUC: 0.9949633699633699
F1: [0.9902912621359222, 0.9767441860465117]
Precision: [1.0, 0.9545454545454546]
Recall: [0.9807692307692307, 1.0]


In [10]:
for feature, importance in zip(['A', 'B', 'C', 'D'], rfc_model.featureImportances):
    print(f'Feature {feature} has importance of {importance * 100:.2f}%')

Feature A has importance of 1.67%
Feature B has importance of 2.20%
Feature C has importance of 94.14%
Feature D has importance of 1.99%
